# Imports

In [1]:
import numpy as np
import tensorflow.keras as keras
from tensorflow.keras.initializers import Zeros
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt
import sklearn.datasets as datasets
import pickle
from mpl_toolkits.mplot3d import Axes3D

# Creating Dataset

the labels, `y`, can be set to 0 or 1 arbitrarily. This attempts to create a dataset that shows off the model.

In [2]:
np.random.seed(1)
n_samples = 250
X, _ = np.random.uniform(-5, 5, size=(n_samples, 2)), None 

layer1_feature = (np.sin(X[:, 0] * 0.5) + np.cos(X[:, 1] * 0.5)) > 0
layer2_feature = (X[:, 0] ** 2 + X[:, 1] ** 2) > 15
y = (layer1_feature ^ layer2_feature).astype(int)

# Model

Input (2D) -> 2 Hidden Layers (3D) -> Output

The hidden_layer_X_model is used while training to save the hidden spaces from epoch to epoch

In [3]:
# Build the model using Input layer
inputs = keras.layers.Input(shape=(2,))
hidden1 = keras.layers.Dense(3, activation="sigmoid", name="hidden_layer_1")(inputs)
hidden2 = keras.layers.Dense(3, activation="sigmoid", name="hidden_layer_2")(hidden1)
outputs = keras.layers.Dense(1, activation="sigmoid", name="output_layer")(hidden2)
model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-1), loss="binary_crossentropy")

hidden_space_trajectory = []
original_points = X  # Store the original points

hidden_layer_1_model = keras.Model(inputs=model.input, outputs=model.get_layer("hidden_layer_1").output)
hidden_layer_2_model = keras.Model(inputs=model.input, outputs=model.get_layer("hidden_layer_2").output)

2025-03-16 19:06:22.821269: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2025-03-16 19:06:22.821350: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2025-03-16 19:06:22.821363: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
I0000 00:00:1742166382.821915   66837 pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
I0000 00:00:1742166382.822519   66837 pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


# Training

This trains the model 1 epoch at a time, and for every interval defined in `epochs_per_interval` records the hidden spaces into `hidden_space_trajectory` as a dictionary for the two layers.

In [4]:
# Train the model for all epochs and record the hidden space every 10 epochs
epochs_per_interval = 10
total_epochs = 300
hidden_space_trajectory = []

for epoch in range(1, total_epochs + 1):
    # Train for one epoch at a time to monitor progress
    model.fit(X, y, epochs=1, verbose=1)

    # Record the hidden space trajectory every 10 epochs
    if epoch % epochs_per_interval == 0:
        # Inspect weights
        weights_hidden1 = model.get_layer("hidden_layer_1").get_weights()[0]
        weights_hidden2 = model.get_layer("hidden_layer_2").get_weights()[0]
        print(f"Weights of hidden layer 1 after {epoch} epochs:\n{weights_hidden1}")
        print(f"Weights of hidden layer 2 after {epoch} epochs:\n{weights_hidden2}")

        hidden1_positions = hidden_layer_1_model.predict(X)
        hidden2_positions = hidden_layer_2_model.predict(X)
        hidden_space_trajectory.append({
            "hidden_layer_1": hidden1_positions,
            "hidden_layer_2": hidden2_positions
        })
        print(f"Recorded hidden space at epoch {epoch}")

# Convert hidden_space_trajectory to numpy array for storage
hidden_space_trajectory = np.array(hidden_space_trajectory)
print(f"Hidden space trajectory shape: {hidden_space_trajectory.shape}")

2025-03-16 19:06:25.570921: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.6551 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.6635 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.6027 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.6116 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.5768 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.5512 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.5263 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.4761 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.4808 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.5453 
Weights of hidden layer 1 after 10 epochs:
[[-1.144701    1.2476614   0.902852  ]
 [-0.12493989  0.11268425  0.11386354]]
Weights of hidden layer 2 after 10 epochs:
[[ 3.4944792  -2.8708928  -1.5694201 ]
 [-2.5343785   3.4790518  -0.5485902 ]
 [-0.8495951   0.19643408 -0.30381888]]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
Recorded hidden space at epoch 10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.4973 
8/8 ━━━━━━━━━━━━━━━━━━━

# Pickling

This gets the hidden spaces, and pickles them with the original points, and labels for use in blender.

In [6]:
# Save the trajectory and original points to a pickle file
with open("hidden_space_and_points.pkl", "wb") as f:
    pickle.dump({
        "hidden_space_trajectory": hidden_space_trajectory, 
        "original_points": original_points, 
        "labels": y},
        f)

In [7]:
#debug-this is just to confirm data is being pickled

for epoch_index, hidden_layers in enumerate(hidden_space_trajectory):
    current_frame =  epoch_index

    print(hidden_layers["hidden_layer_1"])

[[9.89199221e-01 5.87625569e-03 2.64233425e-02]
 [9.99945164e-01 2.03268446e-05 3.90876550e-04]
 [9.99774158e-01 9.99651966e-05 1.15571590e-03]
 [9.99513149e-01 2.17658104e-04 2.20046169e-03]
 [9.93144989e-01 3.72788054e-03 1.80522408e-02]
 [9.89400208e-01 5.80395991e-03 2.59046704e-02]
 [9.98834431e-01 4.97566012e-04 4.74338094e-03]
 [9.99881387e-01 4.32512570e-05 7.60085124e-04]
 [9.91128325e-01 4.91982559e-03 2.21348498e-02]
 [9.99760449e-01 1.04014769e-04 1.23068178e-03]
 [4.53887850e-01 4.84006047e-01 5.34949899e-01]
 [9.96789515e-01 1.57009833e-03 1.02117918e-02]
 [2.77107686e-01 6.89288080e-01 6.76803529e-01]
 [9.99895692e-01 4.35923830e-05 6.23389788e-04]
 [9.99215484e-01 3.23101791e-04 3.47456313e-03]
 [9.99804318e-01 7.91490675e-05 1.08553888e-03]
 [1.91513598e-01 8.03190231e-01 7.43339479e-01]
 [8.67240012e-01 1.03601187e-01 1.69929996e-01]
 [7.84102917e-01 1.62470788e-01 2.60458767e-01]
 [9.99881983e-01 4.22364174e-05 7.66621844e-04]
 [1.12709545e-01 8.84427845e-01 8.303110